In [ ]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from pystac_client import Client
# import ProcessBuilder functions


In [12]:
path_station_details = "/home/khanalp/code/PhD/preprocessICOSdata/csvs/02_station_with_elevation_heightcanopy.csv"
path_sentinel2 = "/home/khanalp/data/sentinel/l2a/"
output_location = "/home/khanalp/data/sentinel/l2a/metadata/"

In [3]:
# Connect to the CDSE STAC catalog
catalog = Client.open("https://catalogue.dataspace.copernicus.eu/stac")

In [4]:
df_station_details = pd.read_csv(path_station_details)

In [5]:
def get_bbox_polygon(latitude: float, longitude: float, buffer_m: float) -> dict:
    """
    Return a GeoJSON polygon representing a square buffer around a point.

    Parameters:
        latitude (float): Latitude of center point.
        longitude (float): Longitude of center point.
        buffer_m (float): Buffer distance in meters.

    Returns:
        dict: GeoJSON polygon.
    """
    km2deg = 1.0 / 111  # Approx. conversion from km to degrees
    buffer_deg = (buffer_m / 1000.0) * km2deg

    west = longitude - buffer_deg
    east = longitude + buffer_deg
    south = latitude - buffer_deg
    north = latitude + buffer_deg

    return {
        "type": "Polygon",
        "coordinates": [[
            [west, south],
            [west, north],
            [east, north],
            [east, south],
            [west, south]  # Close the polygon
        ]]
    }


In [6]:
from datetime import datetime, date

def split_time_range_by_six_months(time_range: list[str]) -> list[tuple[str, str]]:
    start = datetime.strptime(time_range[0], "%Y-%m-%d").date()
    end = datetime.strptime(time_range[1], "%Y-%m-%d").date()

    result = []
    current_start = start

    # First handle the case where we are in the first half of the first year
    if current_start.month <= 6:
        current_end = date(current_start.year, 6, 30)  # End of June
        result.append((current_start.isoformat(), current_end.isoformat()))
        current_start = date(current_start.year, 7, 1)  # Start of second half
    else:
        current_end = date(current_start.year, 12, 31)  # End of December
        result.append((current_start.isoformat(), current_end.isoformat()))
        current_start = date(current_start.year + 1, 1, 1)  # Move to next year

    # Loop for the remaining full years
    while current_start.year < end.year:
        # First half of the year (Jan-Jun)
        current_end = date(current_start.year, 6, 30)
        result.append((current_start.isoformat(), current_end.isoformat()))
        current_start = date(current_start.year, 7, 1)  # Start of second half
        
        # Second half of the year (Jul-Dec)
        current_end = date(current_start.year, 12, 31)
        result.append((current_start.isoformat(), current_end.isoformat()))
        current_start = date(current_start.year + 1, 1, 1)  # Move to next year

    # Handle the last partial period (if any)
    if current_start <= end:
        result.append((current_start.isoformat(), end.isoformat()))

    return result

# Example usage
time_range = ["2015-07-04", "2024-03-31"]
six_month_ranges = split_time_range_by_six_months(time_range)
for r in six_month_ranges:
    print(r)


('2015-07-04', '2015-12-31')
('2016-01-01', '2016-06-30')
('2016-07-01', '2016-12-31')
('2017-01-01', '2017-06-30')
('2017-07-01', '2017-12-31')
('2018-01-01', '2018-06-30')
('2018-07-01', '2018-12-31')
('2019-01-01', '2019-06-30')
('2019-07-01', '2019-12-31')
('2020-01-01', '2020-06-30')
('2020-07-01', '2020-12-31')
('2021-01-01', '2021-06-30')
('2021-07-01', '2021-12-31')
('2022-01-01', '2022-06-30')
('2022-07-01', '2022-12-31')
('2023-01-01', '2023-06-30')
('2023-07-01', '2023-12-31')
('2024-01-01', '2024-03-31')


In [ ]:
for station in df_station_details.station_name:
    all_items = []  
    print(station)  # Print the current station name

    # Filter the DataFrame to get metadata for the current station
    station_data = df_station_details[df_station_details.station_name == station]

    # Extract the latitude and longitude values
    latitude = station_data.latitude.values[0]
    longitude = station_data.longitude.values[0]

    # Create a bounding box polygon (GeoJSON) with 100m buffer around the station
    bbox = get_bbox_polygon(latitude, longitude, 100)
    
    file = [f for f in os.listdir(path_sentinel2) if station in f and f.endswith(".nc")]
    ds_sentinel2 = xr.open_dataset(os.path.join(path_sentinel2, file[0]))
    time = ds_sentinel2.t.values
    coords = bbox['coordinates'][0]

    lons = [pt[0] for pt in coords]
    lats = [pt[1] for pt in coords]

    box = [min(lons), min(lats), max(lons), max(lats)]
    for date_start, date_end in six_month_ranges:
        print(f"Searching for data between {date_start} and {date_end}")
        
        # Search for Sentinel-2 L2A data
        search = catalog.search(
            collections=["SENTINEL-2"],
            bbox=box,
            datetime=f"{date_start}/{date_end}",  # Feed each yearly range here
        )

        items = search.item_collection()
        items = sorted(
            {item.id: item for item in items}.values(),
            key=lambda x: x.properties["datetime"]
        )
        for item in items:
            all_items.append({
                "station": station,
                "datetime": item.properties["datetime"],
                "platform": item.properties.get("platformSerialIdentifier"),

            })
    df_all_items = pd.DataFrame(all_items)
    output_filename = f"{station}_{time_range[0]}_{time_range[1]}.csv"
    output_path = os.path.join(output_location, output_filename)
    df_all_items.to_csv(output_path, index=False)
    # remove this if you want to process all stations

BE-Lon
Searching for data between 2015-07-04 and 2015-12-31
Searching for data between 2016-01-01 and 2016-06-30
Searching for data between 2016-07-01 and 2016-12-31


KeyboardInterrupt: 

In [ ]:
output_filename = f"{station}_{time_range[0]}_{time_range[1]}.csv"
output_path = os.path.join(output_location, output_filename)

In [14]:
output_path


'/home/khanalp/data/sentinel/l2a/metadata/BE-Lon_2015-07-04_2024-03-31.csv'

In [15]:

import os 
import logging  # For logging steps and errors
import openeo  # OpenEO Python client for interacting with back-end processing services
import pandas as pd  # For handling tabular station information

In [16]:
connection = openeo.connect("openeofed.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [17]:
# List collections available on the openEO back-end
connection.list_collection_ids()

['SENTINEL3_OLCI_L1B',
 'SENTINEL3_SLSTR',
 'SENTINEL_5P_L2',
 'COPERNICUS_VEGETATION_PHENOLOGY_PRODUCTIVITY_10M_SEASON1',
 'COPERNICUS_VEGETATION_PHENOLOGY_PRODUCTIVITY_10M_SEASON2',
 'ESA_WORLDCOVER_10M_2021_V2',
 'COPERNICUS_VEGETATION_INDICES',
 'SENTINEL2_L1C',
 'SENTINEL2_L2A',
 'SENTINEL1_GRD',
 'COPERNICUS_30',
 'LANDSAT8_L2',
 'SENTINEL3_SYN_L2_SYN',
 'SENTINEL3_SLSTR_L2_LST',
 'SENTINEL1_GLOBAL_MOSAICS',
 'ESA_WORLDCEREAL_ACTIVECROPLAND',
 'ESA_WORLDCEREAL_IRRIGATION',
 'ESA_WORLDCEREAL_TEMPORARYCROPS',
 'ESA_WORLDCEREAL_WINTERCEREALS',
 'ESA_WORLDCEREAL_MAIZE',
 'ESA_WORLDCEREAL_SPRINGCEREALS',
 'CGLS_GDMP300_V1_GLOBAL',
 'CGLS_GDMP_V2_GLOBAL',
 'CGLS_LAI300_V1_GLOBAL',
 'CGLS_FAPAR300_V1_GLOBAL',
 'CGLS_NDVI300_V2_GLOBAL',
 'SENTINEL3_SYNERGY_VG1',
 'SENTINEL3_SYNERGY_VG10',
 'TERRASCOPE_S2_FAPAR_V2',
 'TERRASCOPE_S2_NDVI_V2',
 'TERRASCOPE_S2_LAI_V2',
 'PROBAV_L3_S10_TOC_333M',
 'PROBAV_L3_S5_TOC_100M',
 'PROBAV_L3_S1_TOC_100M',
 'PROBAV_L3_S1_TOC_333M',
 'TERRASCOPE_S5P_L3

In [19]:
df = pd.read_csv("/home/khanalp/STEMMUS_SCOPE_model/STEMMUS_SCOPE_old/STEMMUS_SCOPE/ICOS_sites/DE-Tha/output/DE-Tha_2024-04-23-1419/reflectance.csv", skiprows=1)

In [20]:
df

fraction of radiation in observation direction *pi / irradiance
NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN 